# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](example.png)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](passed.png)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [1]:
import glob
import os
import numpy as np
import scipy as sp
import scipy.io
import scipy.signal
import matplotlib.pyplot as plt
import pandas as pd

fs=125

def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def BandpassFilter(signal, fs=125):
    '''
    Bandpass filter to remove frequencies between 40BPM and 240 BPM
    '''
    
    b, a = sp.signal.butter(5, (40/60, 240/60), btype='bandpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)


def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)


def RunPulseRateAlgorithm(data_fl, ref_fl):
    """
     Estimates pulse rate
     
     Args:
        data_fl: (str) which is file path string to a troika.mat file with ECG, PPG, and ACC sensor datas.
        ref_fl:  (str) which is a file path to a troika.ma file with reference to heart rate 
     
     Returns:
         errors: a numpy array of errors between pulse rate estimates and corresponding reference heart rates
         confidence: a numpy array of confidence estimates for each pulse rate.
    """
    
   # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)

    # Compute pulse rate estimates 
    
    #Load PPG and Accelerometer Signals.  Get Freqs and FFT Spectra for PPG and Accelerometer signals.
    fs=125
    nfft_window = fs*8
    noverlap = fs*6
    
    ppg_f = BandpassFilter(ppg)
    ppg_f_s, ppg_f_freqs,_,ppg_im = plt.specgram(ppg_f, NFFT = nfft_window, Fs=fs, noverlap=noverlap)
    plt.close()
    
    accx_f = BandpassFilter(accx)
    accx_s, accx_fqs,_,_ = plt.specgram(accx_f, NFFT = nfft_window, Fs=fs, noverlap=noverlap)
    plt.close()
    
    accy_f = BandpassFilter(accy)
    accy_s, accy_fqs,_,_ = plt.specgram(accy_f, NFFT = nfft_window, Fs=fs, noverlap=noverlap)
    plt.close()
    
    accz_f = BandpassFilter(accz)
    accz_s, accz_fqs,_,_ = plt.specgram(accz_f, NFFT = nfft_window, Fs=fs, noverlap=noverlap)
    plt.close()
      
  
    # Estimation confidence.

    max_f_ppg = []
    bpm_d = 10
    distance_bps = bpm_d/60
    count=0
    inner=0
    for i in range(ppg_f_s.shape[1]):
        accx_max_freq = accx_fqs[np.argmax(accx_s[:,i])]
        accy_max_freq = accy_fqs[np.argmax(accy_s[:,i])]
        accz_max_freq = accz_fqs[np.argmax(accz_s[:,i])]
        sorted_ppg_spcs = np.sort(ppg_f_s[:,i])[::-1]
        count += 1
        for f in range(10):
            ppg_freq = ppg_f_freqs[np.argwhere(ppg_f_s == (sorted_ppg_spcs[f]))[0][0]]
            inner+=1
            if ppg_freq == 0:
                continue
            elif (np.abs(ppg_freq-accx_max_freq)<=distance_bps) or (np.abs(ppg_freq-accy_max_freq)<=distance_bps) or (np.abs(ppg_freq-accz_max_freq)<=distance_bps):
                
                if f == 9:
                    max_f_ppg.append(ppg_f_freqs[np.argwhere(ppg_f_s == (sorted_ppg_spcs[0]))[0][0]])
                continue
            else:
                max_f_ppg.append(ppg_freq)
                break
                
                
    bpm_sum_window = 10
    bps_sum_window = bpm_sum_window/60
    
    ecgdata = sp.io.loadmat(ref_fl)['BPM0']
    
    confidences = []
    for i in range(ppg_f_s.shape[1]):
        low_window = max_f_ppg[i]-bps_sum_window 
        high_window = max_f_ppg[i] + bps_sum_window
        window = (ppg_f_freqs>= low_window) & (ppg_f_freqs<=high_window)
        confidence=np.sum(ppg_f_s[:,i][window])/np.sum(ppg_f_s[:,i])
        error = np.abs(max_f_ppg[i]*60-ecgdata[i][0])
        confidences.append((i, max_f_ppg[i]*60, ecgdata[i][0], confidence, error))

    confidence_df = pd.DataFrame(
                    data=confidences, 
                    columns=['WindowNumber','Estimated_Pulse_Rate','Ref_BPM','Confidence','Error'])
    
    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
  
    errors = confidence_df['Error'].values
    confidence = confidence_df['Confidence'].values
    
    return errors, confidence